<a href="https://colab.research.google.com/github/simransirsat/MLPracticals/blob/main/Naive_Continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/sample_data/NaiveBayesContinuous - Sheet1(1).csv')

def testing_split():
    test_df = df.iloc[-2:, :]
    train_df = df.iloc[:-2, :]
    return train_df, test_df
    
train, test = testing_split()
print('Training Dataset:\n', train)
print('Testing Dataset:\n', test)

distr = {}
columns = list(train.columns[:-1])
for i in range(0, train.shape[1] -1):
        # Iterating over all features to create target probability dataframes
    cols = list(train.Target.unique())  # setosa, versicolor, virginica
    temp = pd.DataFrame(columns = cols)
    for j in range(0, temp.shape[1]):
        temp.loc['mu',cols[j]] = round(train[train['Target'] == cols[j]].mean()[i], 4)
        temp.loc['sigma',cols[j]] = round(train[train['Target'] == cols[j]].std()[i], 4)
    distr[columns[i]] = temp

print('Distribution:\n')
for attr,table in distr.items():
    print('Attribute:', attr)
    print(table)

from math import sqrt
from math import exp
def predict():
    columns = list(train.columns[:-1])
    cols = list(train.Target.unique())
    for i in range(0, test.shape[0]):                # Test case
        print('\nTest Case ', i+1,':')
        
        max_likelihood = float('-inf')
        max_likelihood_target = None
        likelihoods = []
        
        for label in cols:                           # Every possible target label
            print('\nLikelihood for ',label,' = P(',label,') * P(',columns[i],'=',test.iloc[i,0],'/',label,')',sep='',end='')
            print()
            for j in range(1, test.shape[1] -1):    
                print('*P(',columns[j],'=', test.iloc[i,j],'/',label,')', sep='',end='')
            print('=',sep='',end='')
            prob = train['Target'].value_counts()[label] / train.shape[0]
            print(prob, sep='', end='')
            
            for j in range(0, test.shape[1] -1): 
                x = test.iloc[i,j]
                mu = distr[columns[j]].loc['mu',label]
                sigma = distr[columns[j]].loc['sigma',label]
                if sigma == 0:
                    continue
                #print('x = ',x,' mu=', mu,'sigma=',sigma)
                prob *= exp(-((x - mu)**2) / (2 * (sigma ** 2))) / (sqrt(2 * 3.14) * sigma)
                print(' \n * exp(-((',x,' - ',mu,')**2) / (2 * (',sigma,' ** 2))) / (sqrt(2 * 3.14) * ',sigma,')) ')
                #print('[',round(exp(-((x - mu)**2) / (2 * (sigma ** 2))) / (sqrt(2 * 3.14) * sigma), 4),']', sep='',end='')
            
            print('= ',train['Target'].value_counts()[label] / train.shape[0],sep='',end='')
            for j in range(0, test.shape[1] -1):
                x = test.iloc[i,j]
                mu = distr[columns[j]].loc['mu',label]
                sigma = distr[columns[j]].loc['sigma',label]
                if sigma == 0:
                    continue
                print('*',exp(-((x - mu)**2) / (2 * (sigma ** 2))) / (sqrt(2 * 3.14) * sigma), sep='',end='')    
            
            print('= ',prob)
            
            likelihoods.append(prob)
            
            if(prob>max_likelihood):
                max_likelihood = prob
                max_likelihood_target = label
                
        print(likelihoods)
        print('Predicted: ', max_likelihood_target, 'Original: ', test.iloc[i,-1])
        if(test.iloc[i, -1] == max_likelihood_target):
            print('Classified Correctly')
        else:
            print('Classified Incorrectly')

predict()

Training Dataset:
      sl   sw   pl   pw      Target
0   5.6  2.8  4.9  2.0   virginica
1   6.9  3.1  4.9  1.5  versicolor
2   6.1  2.8  4.7  1.2  versicolor
3   5.8  2.7  3.9  1.2  versicolor
4   5.7  2.8  4.5  1.3  versicolor
5   6.0  2.7  5.1  1.6  versicolor
6   5.4  3.4  1.5  0.4      setosa
7   7.3  2.9  6.3  1.8   virginica
8   4.8  3.4  1.9  0.2      setosa
9   4.8  3.1  1.6  0.2      setosa
10  5.1  3.5  1.4  0.2      setosa
11  5.5  2.4  3.8  1.1  versicolor
12  5.1  3.8  1.5  0.3      setosa
Testing Dataset:
      sl   sw   pl   pw      Target
13  5.4  3.7  1.5  0.2      setosa
14  6.6  3.0  4.4  1.4  versicolor
Distribution:

Attribute: sl
      virginica versicolor setosa
mu         6.45          6   5.04
sigma    1.2021     0.4899  0.251
Attribute: sw
      virginica versicolor setosa
mu         2.85       2.75   3.44
sigma    0.0707     0.2258  0.251
Attribute: pl
      virginica versicolor  setosa
mu          5.6     4.4833    1.58
sigma    0.9899     0.5307  0.1924
At